In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import pi
import cv2
import scipy.misc
import tensorflow as tf

DATA_FOLDER = r"G:\VIT\Fallsem 2019\NNDL\Project\Self-Driving-Car-master\data1\data"
DATA_FILE = os.path.join(DATA_FOLDER, "data.txt")

x = []
y = []

train_batch_pointer = 0
test_batch_pointer = 0

with open(DATA_FILE) as f:
    
    for line in f:
        arb, time = line.split(',')
        image_name, angle = arb.split()
        
        image_path = os.path.join(DATA_FOLDER, image_name)
        x.append(image_path)
        
        angle_radians = float(angle) * (pi / 180)  #converting angle into radians
        y.append(angle_radians)
y = np.array(y)

split_ratio = int(len(x) * 0.8)

train_x = x[:split_ratio]
train_y = y[:split_ratio]

test_x = x[split_ratio:]
test_y = y[split_ratio:]

def weightVariable(shape):
    initial = tf.truncated_normal(shape = shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def convolution(previous_input, filter_input, strides):
    return tf.nn.conv2d(previous_input, filter_input, strides = [1, strides, strides, 1], padding = "VALID")

x_input = tf.placeholder(tf.float32, shape = [None, 66, 200, 3], name = "Plc_1")
y_true = tf.placeholder(tf.float32, name = "Plc_2")

input_image = x_input

#Convolution Layers
#First convolution layer
W_Conv1 = weightVariable([5,5,3,24])
B_Conv1 = bias_variable([24])
Conv1 = tf.nn.relu(convolution(input_image, W_Conv1, 2) + B_Conv1)
#strides = 2
#Output size: 31*98*24

#Second convolution layer
W_Conv2 = weightVariable([5,5,24,36])
B_Conv2 = bias_variable([36])
Conv2 = tf.nn.relu(convolution(Conv1, W_Conv2, 2) + B_Conv2)
#strides = 2
#Output size: 14*47*36

#Third convolution layer
W_Conv3 = weightVariable([5,5,36,48])
B_Conv3 = bias_variable([48])
Conv3 = tf.nn.relu(convolution(Conv2, W_Conv3, 2) + B_Conv3)
#strides = 2
#Output size: 5*22*48

#Fourth convolution layer
W_Conv4 = weightVariable([3,3,48,64])
B_Conv4 = bias_variable([64])
Conv4 = tf.nn.relu(convolution(Conv3, W_Conv4, 1) + B_Conv4)
#strides = 1
#Output size: 3*20*64


#Fifth convolution layer
W_Conv5 = weightVariable([3,3,64,64])
B_Conv5 = bias_variable([64])
Conv5 = tf.nn.relu(convolution(Conv4, W_Conv5, 1) + B_Conv5)
#strides = 1
#Output size: 1*18*64

#Fully-Connected Dense Layers
keep_prob = tf.placeholder(tf.float32)
#First FC-Dense
#Input = 1*18*64 = 1152
W_FC1 = weightVariable([1152, 1164])
B_FC1 = bias_variable([1164])
FC1_Flatten = tf.reshape(Conv5, [-1, 1152]) #here, -1 indicates 1. It means that the shape of FC1_Flatten will be 1*1152
Output_FC1 = tf.nn.relu(tf.matmul(FC1_Flatten, W_FC1) + B_FC1) #so, here shape of FC1_Flatten is 1*1152 and shape of W_FC1 will
#be 1152*1164. Therefore, there will be a matrix multiplication of matrices: (1*1152) * (1152*1164) = (1*1164).
Output_FC1_drop = tf.nn.dropout(Output_FC1, keep_prob)

#Second FC-Dense
#Input = 1*1164 = 1164
W_FC2 = weightVariable([1164, 100])
B_FC2 = bias_variable([100])
Output_FC2 = tf.nn.relu(tf.matmul(Output_FC1_drop, W_FC2) + B_FC2) #so, here shape of Output_FC1_drop is 1*1164 and shape of 
#W_FC2 will be 1164*100. Therefore, there will be a matrix multiplication of matrices: (1*1164) * (1164*100) = (1*100).
Output_FC2_drop = tf.nn.dropout(Output_FC2, keep_prob)

#Third FC-Dense
#Input = 1*100 = 100
W_FC3 = weightVariable([100, 50])
B_FC3 = bias_variable([50])
Output_FC3 = tf.nn.relu(tf.matmul(Output_FC2_drop, W_FC3) + B_FC3) #so, here shape of Output_FC2_drop is 1*100 and shape of 
#W_FC3 will be 100*50. Therefore, there will be a matrix multiplication of matrices: (1*100) * (100*50) = (1*50).
Output_FC3_drop = tf.nn.dropout(Output_FC3, keep_prob)

#Fourth FC-Dense
#Input = 1*50 = 50
W_FC4 = weightVariable([50, 10])
B_FC4 = bias_variable([10])
Output_FC4 = tf.nn.relu(tf.matmul(Output_FC3_drop, W_FC4) + B_FC4) #so, here shape of Output_FC3_drop is 1*50 and shape of 
#W_FC4 will be 50*10. Therefore, there will be a matrix multiplication of matrices: (1*50) * (50*10) = (1*10).
Output_FC4_drop = tf.nn.dropout(Output_FC4, keep_prob)

#Final Output to one neuron with linear/identity function
#Input = 1*10 = 10
W_FC5 = weightVariable([10, 1])
B_FC5 = bias_variable([1])
y_predicted = tf.identity(tf.matmul(Output_FC4_drop, W_FC5) + B_FC5)

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, r"G:\VIT\Fallsem 2019\NNDL\Project\Self-Driving-Car-master\data1\saver\model.ckpt")

img = cv2.imread(r'G:\VIT\Fallsem 2019\NNDL\Project\Self-Driving-Car-master\Self-Driving-Car-master\SelfDrivingCar\steering_wheel_image.jpg', 0)
rows, cols = img.shape

i = 0

while(cv2.waitKey(50) != ord("q")):
    full_image = cv2.imread(test_x[i])
    cv2.imshow('Frame Window', full_image)
    image = ((cv2.resize(full_image[-150:], (200, 66)) / 255.0).reshape((1, 66, 200, 3)))
    degrees = sess.run(y_predicted, feed_dict = {x_input: image, keep_prob: 1.0})[0][0] *180 / pi
    print("Predicted degrees: "+str(degrees))
    M = cv2.getRotationMatrix2D((cols/2,rows/2), -degrees, 1)
    dst = cv2.warpAffine(src = img, M = M, dsize = (cols, rows))
    cv2.imshow("Steering Wheel", dst)
    i += 1

cv2.destroyAllWindows()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from G:\VIT\Fallsem 2019\NNDL\Project\Self-Driving-Car-master\data1\saver\model.ckpt
Predicted degrees: 21.77694826284553
Predicted degrees: 19.436764943753666
Predicted degrees: 23.05672763781999
Predicted degrees: 21.348828590574545
Predicted degrees: 22.652404344617327
Predicted degrees: 21.980962598259225
Predicted degrees: 22.6230516066592
Predicted degrees: 20.648601011959425
Predicted degrees: 16.851630550457635
Predicted degrees: 14.149973983694832
Predicted degrees: 11.764427963230329
Predicted degrees: 12.070617659759185
Predicted degrees: 10.631636820515483
Predicted degrees: 10.58807216644185
Predicted degrees: 9.386327702734786
Predicted degrees: 11.290565660314122
Pr

Predicted degrees: 13.34715696374612
Predicted degrees: 13.826649903859353
Predicted degrees: 12.973827361449684
Predicted degrees: 13.168399814109483
Predicted degrees: 11.746072683609565
Predicted degrees: 12.077217330044709
Predicted degrees: 11.858952658701428
Predicted degrees: 5.2541157619563155
Predicted degrees: 0.869927043638674
Predicted degrees: 2.1395275157591103
Predicted degrees: -0.3883812047930269
Predicted degrees: -1.48408527408835
Predicted degrees: -1.2877706963033742
Predicted degrees: -0.670537600066723
Predicted degrees: -1.3363837139472936
Predicted degrees: 0.46318671724347593
Predicted degrees: -1.5092703159923027
Predicted degrees: -0.18463965005930844
Predicted degrees: -1.2063556951704684
Predicted degrees: -2.444352241671021
Predicted degrees: 0.30874290661797077
Predicted degrees: 3.0790095773671973
Predicted degrees: 1.0407680040272642
Predicted degrees: 1.3448121673830893
Predicted degrees: -0.7073578692217924
Predicted degrees: -1.3564683120885392
Pred

Predicted degrees: 9.040527177888087
Predicted degrees: 8.109268450597284
Predicted degrees: 7.354039919916988
Predicted degrees: 7.0696035221209765
Predicted degrees: 7.875852712127622
Predicted degrees: 5.799304022827672
Predicted degrees: 3.8339461174604756
Predicted degrees: 7.1543170777361755
Predicted degrees: 5.707311193104532
Predicted degrees: 7.527864819199227
Predicted degrees: 7.479658197810924
Predicted degrees: 11.239420350035422
Predicted degrees: 8.096787133662732
Predicted degrees: 8.57082446017642
Predicted degrees: 8.290927576857529
Predicted degrees: 6.708879887711447
Predicted degrees: 6.265577131802347
Predicted degrees: 7.180275638563632
Predicted degrees: 4.926435728977661
Predicted degrees: 5.67142879429987
Predicted degrees: 4.805906795789031
Predicted degrees: 5.502183109969301
Predicted degrees: 4.659685679150575
Predicted degrees: 5.40460574682603
Predicted degrees: 6.661190653152771
Predicted degrees: 5.709005933792342
Predicted degrees: 5.523577396883897


Predicted degrees: 7.613878245190844
Predicted degrees: 10.04393660847632
Predicted degrees: 8.414075887592858
Predicted degrees: 9.685378997807332
Predicted degrees: 10.784414491870328
Predicted degrees: 9.771692952122429
Predicted degrees: 7.081759551296307
Predicted degrees: 12.153286854378434
Predicted degrees: 7.337247899842511
Predicted degrees: 5.095633601984041
Predicted degrees: 6.90205983555421
Predicted degrees: 10.180428547528917
Predicted degrees: 15.191395978335125
Predicted degrees: 16.048654728610718
Predicted degrees: 13.855224854025748
Predicted degrees: 7.921051488960181
Predicted degrees: 14.385619778928504
Predicted degrees: 15.380807369303334
Predicted degrees: 13.629896059533381
Predicted degrees: 11.237028076278625
Predicted degrees: 11.743925442889243
Predicted degrees: 10.005928313292491
Predicted degrees: 8.86336743874359
Predicted degrees: 8.468500542446812
Predicted degrees: 6.565344742964099
Predicted degrees: 5.722294920596248
Predicted degrees: 5.5745694

Predicted degrees: -10.114908250368277
Predicted degrees: -9.091854341953441
Predicted degrees: -10.865047436343268
Predicted degrees: -13.83528326297025
Predicted degrees: -15.910188011114341
Predicted degrees: -14.836246632061972
Predicted degrees: -12.185944550121203
Predicted degrees: -11.515701501962438
Predicted degrees: -11.386499936075182
Predicted degrees: -10.943867819365215
Predicted degrees: -11.734928376205044
Predicted degrees: -14.412901262020826
Predicted degrees: -13.436743432447297
Predicted degrees: -10.247475391969054
Predicted degrees: -10.711798481935691
Predicted degrees: -12.363012081711906
Predicted degrees: -13.419550138759083
Predicted degrees: -13.276278383181603
Predicted degrees: -10.855024133736274
Predicted degrees: -10.295559923725945
Predicted degrees: -11.384531134046927
Predicted degrees: -11.771526237325265
Predicted degrees: -14.601916502017176
Predicted degrees: -12.028416482198613
Predicted degrees: -12.107709855820584
Predicted degrees: -11.5885

...

Predicted degrees: 14.070976015754464
Predicted degrees: 14.740753757276025
Predicted degrees: 13.854630627567957
Predicted degrees: 15.364732519091708
Predicted degrees: 12.097515798484341
Predicted degrees: 15.462984790302341
Predicted degrees: 14.877535125594699
Predicted degrees: 11.644693893306359
Predicted degrees: 17.9115427209548
Predicted degrees: 22.347439813270846
Predicted degrees: 19.398454412699063
Predicted degrees: 17.87686926563323
Predicted degrees: 19.59031442648472
Predicted degrees: 16.945993028935945
Predicted degrees: 23.300647353459485
Predicted degrees: 26.52982077416353
Predicted degrees: 27.305603905377332
Predicted degrees: 24.994677701595215
Predicted degrees: 24.217730023127928
Predicted degrees: 27.325281680376136
Predicted degrees: 31.176613617482058
Predicted degrees: 26.027518417317033
Predicted degrees: 23.37639073626034
Predicted degrees: 25.909800106971865
Predicted degrees: 15.18912152534151
Predicted degrees: 16.733639032545664
Predicted degrees: 

Predicted degrees: -1.2137267499453814
Predicted degrees: -11.051318647840562
Predicted degrees: -11.975668638785898
Predicted degrees: -14.004627558156962
Predicted degrees: -13.484565001921142
Predicted degrees: -11.698424430185911
Predicted degrees: 2.8571801876074043
Predicted degrees: 5.612713926873502
Predicted degrees: -4.344839571622375
Predicted degrees: 6.220790727640928
Predicted degrees: -0.8932615311643763
Predicted degrees: 0.4183226196802472
Predicted degrees: -0.4815240676176677
Predicted degrees: -3.1857274411672387
Predicted degrees: 4.129092678761889
Predicted degrees: 3.799342244691075
Predicted degrees: -8.125772748952974
Predicted degrees: -0.6398905412608759
Predicted degrees: 5.462837805255443
Predicted degrees: 3.587398913424641
Predicted degrees: 8.700665502524716
Predicted degrees: 0.6126479048694569
Predicted degrees: -1.3697517493637446
Predicted degrees: -4.402435141798508
Predicted degrees: -5.411885874707617
Predicted degrees: -13.576793046283825
Predict

Predicted degrees: -11.193050195760204
Predicted degrees: -10.429645074869757
Predicted degrees: -9.083121091325934
Predicted degrees: -9.197006811773079
Predicted degrees: -8.74445299152002
Predicted degrees: -8.394044650304503
Predicted degrees: -8.57328162073033
Predicted degrees: -7.840351096369188
Predicted degrees: -8.358907595893019
Predicted degrees: -8.350798453800921
Predicted degrees: -8.309368233842916
Predicted degrees: -8.219796279293723
Predicted degrees: -8.17212241266068
Predicted degrees: -8.659136244823745
Predicted degrees: -7.615073528295596
Predicted degrees: -7.534281781924454
Predicted degrees: -8.163263657307173
Predicted degrees: -7.760434467985454
Predicted degrees: -7.513169667199945
Predicted degrees: -7.657883446465943
Predicted degrees: -7.872747537376204
Predicted degrees: -7.303984878584549
Predicted degrees: -6.9280674883663265
Predicted degrees: -6.184822524584818
Predicted degrees: -6.077207771569389
Predicted degrees: -6.179506076089608
Predicted de

Predicted degrees: 20.752392566586938
Predicted degrees: 20.804882570358483
Predicted degrees: 21.15295925574609
Predicted degrees: 20.72798830068248
Predicted degrees: 20.491168566685214
Predicted degrees: 19.83650518002325
Predicted degrees: 19.68640835791763
Predicted degrees: 19.33925374806798
Predicted degrees: 18.3517757337188
Predicted degrees: 17.708416310133202
Predicted degrees: 17.763704983917165
Predicted degrees: 17.153651270271844
Predicted degrees: 16.852013041051155
Predicted degrees: 16.390828423008138
Predicted degrees: 15.764175642132395
Predicted degrees: 16.83428870015497
Predicted degrees: 16.908220375278482
Predicted degrees: 16.236616411927592
Predicted degrees: 19.556399122161025
Predicted degrees: 21.075668835098224
Predicted degrees: 21.11230255471159
Predicted degrees: 21.012424698478494
Predicted degrees: 21.238220507155358
Predicted degrees: 19.575219708418995
Predicted degrees: 19.98067339792929
Predicted degrees: 20.125312898888065
Predicted degrees: 20.

Predicted degrees: 13.945377374654393
Predicted degrees: 15.259298303968807
Predicted degrees: 15.264755625115647
Predicted degrees: 15.14538099389675
Predicted degrees: 15.596096639489428
Predicted degrees: 13.435206639884045
Predicted degrees: 9.521386155835325
Predicted degrees: 4.576952024621034
Predicted degrees: 5.772763188373472
Predicted degrees: 6.604166684432805
Predicted degrees: 11.594905229351552
Predicted degrees: 14.014761851337967
Predicted degrees: 14.527692836306645
Predicted degrees: 16.21480761790774
Predicted degrees: 16.81735666120251
Predicted degrees: 16.395879347899363
Predicted degrees: 14.051217132259264
Predicted degrees: 11.975450926506104
Predicted degrees: 8.571866917798493
Predicted degrees: -0.8632257742892453
Predicted degrees: -4.633346762163714
Predicted degrees: -7.790087734267064
Predicted degrees: -9.0825021054324
Predicted degrees: -8.263443749411042
Predicted degrees: -5.523165024212757
Predicted degrees: -3.978247950942158
Predicted degrees: -3